In [ ]:
import os
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate

In [10]:
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city."""
    return f"Weather in {city}: Sunny 25°C"


prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action (a valid JSON object, e.g., {{"city": "Tehran"}})
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""  # Note: {agent_scratchpad} is a string placeholder here

prompt = PromptTemplate.from_template(prompt_template)

In [5]:
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "llama3.2:3b"

max_tokens = 100
llm = ChatOllama(model=model_name, base_url=OLLAMA_URL, num_predict=max_tokens, temperature=0)

In [11]:
agent = create_react_agent(llm=llm, tools=[get_weather], prompt=prompt)
executor = AgentExecutor(agent=agent, tools=[get_weather], verbose=False)

result = executor.invoke({"input": "هوای تهران چطوره؟"})
print(result["output"])

هوای تهران در حال حاضر خنک و خورشید است.
